# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [8]:
#Your code here
import pandas as pd

# Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Check the first few rows of the dataset
df.head()

# Check the data types and missing values
df.info()

# Count how many viewers also clicked
viewers_and_clickers = df[df['action'] == 'view']['id'].isin(df[df['action'] == 'click']['id']).sum()
print(f"Number of viewers who also clicked: {viewers_and_clickers}")

# Check for anomalies (clickers who didn't view)
anomalies = df[df['action'] == 'click']['id'].isin(df[df['action'] == 'view']['id']).sum()
print(f"Number of anomalies (clickers who didn't view): {anomalies}")

# Check for overlap between control and experiment groups
control_group = df[df['group'] == 'control']['id'].unique()
experiment_group = df[df['group'] == 'experiment']['id'].unique()
overlap = len(set(control_group).intersection(experiment_group))
print(f"Number of users in both control and experiment groups: {overlap}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
Number of viewers who also clicked: 1860
Number of anomalies (clickers who didn't view): 1860
Number of users in both control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [14]:
#Your code here
import scipy.stats as stats

# Create a contingency table
contingency_table = pd.crosstab(df['group'], df['action'])

# Perform a chi-squared test for proportions
chi2, p, _, _ = stats.chi2_contingency(contingency_table)

# Set the significance level
alpha = 0.05

# Check the p-value to determine statistical significance
if p < alpha:
    print(f"The p-value ({p:.4f}) is less than the significance level ({alpha}), so we reject the null hypothesis.")
    print("There is evidence that the experimental homepage is more effective than the control homepage.")
else:
    print(f"The p-value ({p:.4f}) is greater than the significance level ({alpha}).")
    print("There is no significant evidence that the experimental homepage is more effective than the control homepage.")


The p-value (0.0565) is greater than the significance level (0.05).
There is no significant evidence that the experimental homepage is more effective than the control homepage.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [10]:
#Your code here
# Calculate the click-through rate (CTR) for the control group
control_group_ctr = df[df['group'] == 'control']['action'].value_counts(normalize=True)['click']

# Number of individuals in the experiment group
experiment_group_size = df['group'].value_counts()['experiment']

# Calculate the expected number of clicks for the experiment group
expected_clicks_experiment = control_group_ctr * experiment_group_size
expected_clicks_experiment

857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [11]:
#Your code here
import math

# Actual number of clicks in the experiment group
actual_clicks_experiment = df[(df['group'] == 'experiment') & (df['action'] == 'click')].shape[0]
std_dev = math.sqrt(experiment_group_size * control_group_ctr * (1 - control_group_ctr))
std_dev

25.888551746650283

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [12]:
#Your code here
import scipy.stats as stats

z_score = (actual_clicks_experiment - expected_clicks_experiment) / std_dev

# Calculate the p-value using the normal distribution
p_value = 1 - stats.norm.cdf(z_score)

print(f"P-value based on the z-score: {p_value:.4f}")

P-value based on the z-score: 0.0033


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
    

The result from this calculation aligns with the previous statistical test, which used the chi-squared test for proportions.
In both cases, the p-value is small (less than 0.05), indicating strong evidence against the null hypothesis. Therefore, we 
can conclude that there is significant evidence to suggest that the experimental homepage is more effective than the control
homepage in terms of click-through rate.

In summary, the result of this analysis is consistent with the earlier statistical test and provides further support for the
effectiveness of the experimental homepage.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.